In [138]:
from bs4 import BeautifulSoup, SoupStrainer
import re
import requests
from sets import Set 
import mechanize
import urllib2

The site you gave me as a starting point does not contain the proposals but several links to the proposals within different time frames, so I decided to start from there as well. The first step is to extract those links, and allow the capability to choose between them. 

In [15]:
r = requests.get("https://www.eso.org/sci/observing/teles-alloc/all.html")

In [17]:
response = r.text

In [31]:
pds = []
soup = BeautifulSoup(response)
for link in soup.find("div", {"class": "par parsys"}).find_all("a"): 
    if "href" in str(link):
        pds.append(link["href"])

In [85]:
print pds[0]

http://archive.eso.org/wdb/wdb/eso/approved_runs/query?period=97&order=prog_id&prog_type=%21%3D2


What we have done is selected the most recent link with regards to time period. This can be updated to choose a link based off of a requested time period. This link takes us to the page in which we can query proposals based off of a string. The query we are going to use randomly here is "supercluster" just because that sounds awesome. 

In [86]:
r2 = requests.get(pds[0])

In [87]:
esosoup = BeautifulSoup(r2.text)

In [89]:
query = "asteroid"

In [96]:
allrows = esosoup.findAll('tr')
userrows = [t for t in allrows if t.findAll(text=re.compile(query))]

In [103]:
ids = []
for row in userrows: 
    ids.append(str(row.a).split(">")[1][:-6])

In [112]:
query_ids = list(Set(ids))

In [114]:
query_ids

['097.C-0853',
 '595.C-0650',
 '097.C-0912',
 '095.C-0087',
 '097.C-0248',
 '097.C-0166']

Now that we have the ids, we need to plug them into the form so that we can get the correct time and coordinates of the observation. 

In [164]:
browser = mechanize.Browser()
browser.set_handle_robots(False)
browser.open("http://archive.eso.org/eso/eso_archive_main.html")
browser.select_form(nr=0)

browser.form['prog_id'] = str(query_ids[1])
req = browser.submit()
final = req.geturl()
x = browser.open(final)
test = BeautifulSoup(x)

In [169]:
rows = test.findAll("tr")
out = [t for t in rows if t.findAll(text=re.compile("DIMM"))]

In [186]:
r = []
for row in out:
    r.append(row.findAll("td")) #row.findAll("td")
r[0]

[<td align="center"><div class="public_mark"> <input name="dataset" type="checkbox" value="SAF+KMOS.2014-06-06T16:12:43.181"/> </div></td>,
 <td align="center"><a href="/wdb/wdb/eso/eso_archive_main/query?dp_id=KMOS.2014-06-06T16:12:43.181&amp;format=&amp;tab_stat_plot=on&amp;aladin_colour=aladin_instrument"><img border="0" height="22" src="/wdb/images/more.gif" width="22"/></a></td>,
 <td><a href="/hdr?DpId=KMOS.2014-06-06T16:12:43.181" target="HDR">Header</a></td>,
 <td>WAVE,LAMP</td>,
 <td>\xa0</td>,
 <td><a href="/wdb/wdb/eso/sched_rep_arc/query?progid=60.A-9254(A)">60.A-9254(A)</a></td>,
 <td>KMOS</td>,
 <td>CALIB</td>,
 <td>WAVE,LAMP</td>,
 <td>SPECTRUM</td>,
 <td>KMOS.2014-06-06T16:12:43.181</td>,
 <td>Jun\xa0\xa06\xa02014</td>,
 <td>KMOS_spec_cal_wave</td>,
 <td>2014-06-06T16:04:50</td>,
 <td align="right">12.000</td>,
 <td>H,H,H</td>,
 <td align="right">56814.675500</td>,
 <td align="right">\xa0</td>,
 <td><a href="http://archive.eso.org/asm/ambient-server?site=paranal&amp;zoo